In [1]:
import numpy as np 
import pandas as pd 
from sklearn.model_selection import train_test_split 
from sklearn.preprocessing import OneHotEncoder, LabelEncoder, StandardScaler 
from sklearn.model_selection import train_test_split, KFold, StratifiedKFold  
from sklearn.preprocessing import OneHotEncoder, LabelEncoder, StandardScaler 
import lightgbm as lgbm
from catboost import CatBoostClassifier 
from xgboost import XGBClassifier

We first setup the pycaret environment 

In [2]:
# import pycaret library for binary classification 
from pycaret.classification import *

In [3]:
#setup train and test dataframes 
train = pd.read_csv('train.csv') 
test = pd.read_csv('test_x.csv') 
submission = pd.read_csv('sample_submission.csv') 

train = train.drop(columns=['index']) 
test = test.drop(columns=['index'])

In [4]:
clf = setup(data = train, target = 'voted')

Setup Succesfully Completed!


,Description,Value
0,session_id,7386
1,Target Type,Binary
2,Label Encoded,"1: 0, 2: 1"
3,Original Data,"(45532, 77)"
4,Missing Values,False
5,Numeric Features,41
6,Categorical Features,35
7,Ordinal Features,False
8,High Cardinality Features,False
9,High Cardinality Method,None


In [5]:
best_7 = compare_models(sort = 'AUC', n_select = 7)

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
0,Gradient Boosting Classifier,0.6973,0.7659,0.6402,0.7678,0.6981,0.3996,0.4063,20.8064
1,CatBoost Classifier,0.6935,0.7658,0.6549,0.7525,0.7002,0.3897,0.3937,11.9484
2,Light Gradient Boosting Machine,0.6958,0.7643,0.6441,0.7628,0.6984,0.3959,0.4018,0.9228
3,Extra Trees Classifier,0.6935,0.7625,0.6430,0.7596,0.6965,0.3912,0.3968,2.1444
4,Linear Discriminant Analysis,0.6929,0.7621,0.6593,0.7490,0.7013,0.3879,0.3912,0.7605
5,Ada Boost Classifier,0.6906,0.7595,0.6504,0.7506,0.6969,0.3843,0.3884,4.6764
6,Extreme Gradient Boosting,0.6777,0.7464,0.6629,0.7245,0.6922,0.3554,0.3571,25.1877
7,Random Forest Classifier,0.6610,0.7160,0.6104,0.7260,0.6632,0.3271,0.3321,0.2642
8,Decision Tree Classifier,0.6106,0.6074,0.6417,0.6448,0.6432,0.2147,0.2148,1.4619
9,Naive Bayes,0.4542,0.5219,0.0100,0.5427,0.0197,0.0001,-0.0003,0.0799


In [6]:
# instead of blending, let us try stacking, using lgbm as a meta-model  
lgbm = create_model('lightgbm') 
stack_soft = stack_models(best_7, meta_model = lgbm) 

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,0.6964,0.7675,0.6363,0.7685,0.6962,0.3981,0.4052
1,0.6932,0.7580,0.6351,0.7640,0.6936,0.3916,0.3983
2,0.6903,0.7639,0.6299,0.7625,0.6899,0.3862,0.3932
3,0.7051,0.7677,0.6265,0.7907,0.6991,0.4174,0.4283
4,0.7007,0.7629,0.6391,0.7741,0.7002,0.4067,0.4142
5,0.6869,0.7579,0.6202,0.7629,0.6842,0.3801,0.3882
6,0.7094,0.7767,0.6489,0.7827,0.7095,0.4239,0.4314
7,0.6818,0.7536,0.6242,0.7519,0.6821,0.3691,0.3755
8,0.6981,0.7726,0.6234,0.7802,0.6930,0.4033,0.4132
9,0.6922,0.7669,0.6303,0.7652,0.6912,0.3900,0.3973


In [8]:
# make predictions on holdout set 
pred_holdout = predict_model(stack_soft)

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,Stacking Classifier,0.6921,0.7611,0.628,0.7668,0.6905,0.3901,0.3979


In [9]:
final_model = finalize_model(stack_soft)
predictions = predict_model(final_model, data = test)

In [11]:
submission['voted'] = predictions['Score'] 
submission.head()

,index,voted
0,0,0.5758
1,1,0.8990
2,2,0.4653
3,3,0.2029
4,4,0.7387


In [12]:
submission.to_csv('stack_lgbm.csv',index=False)

Averaging results from soft blend and soft stack 

In [14]:
blend_soft = pd.read_csv('autoML_probability_7.csv') 
avg_vote = (blend_soft['voted']+submission['voted'])/2.0 

In [16]:
submission['voted'] = avg_vote 
submission.head()

,index,voted
0,0,0.60805
1,1,0.86600
2,2,0.48025
3,3,0.21040
4,4,0.73070


In [17]:
submission.to_csv('stack_blend_ensemble.csv',index=False)